---
# ADAS - Ingest Video and Sensor Streams

To avoid the need for a set of live cameras for this demo, we play back video from a series of JPEG files on disk
and write each video frame to SDP.
These videos are part of the [KITTI Vision Benchmark Suite](http://www.cvlibs.net/datasets/kitti/raw_data.php).

---

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, opencv_image_to_mpl, ImageFileSequenceLoader, UnindexedStream
import os
from pathlib import Path
import pandas as pd
import numpy as np
import base64
from multiprocessing import Process
import json
from itertools import islice
import time

### Define Pravega stream parameters

In [9]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'

### Initialize connection to Pravega GRPC Gateway

In [10]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [11]:
#pravega_client.CreateScope(pravega.pb.CreateScopeRequest(scope=scope))

### Create Pravega stream

In [12]:
streams_to_delete = [
    'adas-raw-video',
    'adas-raw-sensor',
    'object-detector-output-video',
    'object-detector-output-test',
    'adas-raw-video' + '-index',
    'adas-raw-sensor' + '-index',
    'object-detector-output-video' + '-index',
    'object-detector-output-test' + '-index',
]
streams_to_delete

['adas-raw-video',
 'adas-raw-sensor',
 'object-detector-output-video',
 'object-detector-output-test',
 'adas-raw-video-index',
 'adas-raw-sensor-index',
 'object-detector-output-video-index',
 'object-detector-output-test-index']

In [13]:
for s in streams_to_delete: UnindexedStream(pravega_client, scope, s).create_stream(min_num_segments=1)

In [14]:
for s in streams_to_delete: UnindexedStream(pravega_client, scope, s).truncate_stream()

In [16]:
for s in streams_to_delete: UnindexedStream(pravega_client, scope, s).create_stream(min_num_segments=1)